## Particle in Cell method for the one-dimensional two-counterstreaming plasma

In [ ]:
import numpy as np
import scipy.linalg as la

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
%%capture
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['animation.embed_limit'] = 2**64

fig, ax = plt.subplots()
colors = np.array([('r' if i%2==0 else 'b') for i in range(setupObj.numParticles)])
scatterPlot = ax.scatter(0, 0, s=5.)

ax.set_xlim(0, length)
ax.set_ylim(-initStreamVelocity*8, initStreamVelocity*8)
dynamicAxisLimits = False
axisLimitWspace = 1.2
minVelocities = np.min(velocities, axis=1) * axisLimitWspace
minVelocities = minVelocities * (minVelocities<0)
maxVelocities = np.max(velocities, axis=1) * axisLimitWspace
maxVelocities = maxVelocities * (maxVelocities>0)

def animateStep(frame):
    scatterPlot.set_offsets(np.vstack([positions[frame],velocities[frame]]).T)
    scatterPlot.set_color(colors)
    if dynamicAxisLimits and frame%10==0:
        ax.set_ylim(minVelocities[frame], maxVelocities[frame])
    return scatterPlot,

Define the normalized/unitless constants

In [ ]:
class Setup:
    def __init__(self, l, ns, nc, np, dt, qm):
        self.length = l
        self.numSteps = ns
        self.numCells = nc
        self.numParticles = np
        self.cellSize = l/nc
        self.timeStep = dt
        self.qm_ratio = qm

Setting up the arrays that will store the charge density, potential, and electric field in each cell; as well as arrays for storing particle positions and velocities. Assigning initial values for these quantities.

In [ ]:
setupObj = Setup(1., 2000, 50, 800, 0.007, 0.02)
potentials = np.zeros((setupObj.numSteps, setupObj.numCells))
elfields = np.zeros((setupObj.numSteps, setupObj.numCells))
chargedensities = np.zeros((setupObj.numSteps, setupObj.numCells))

positions = np.zeros((setupObj.numSteps, setupObj.numParticles))
positions[0] = np.linspace(0, setupObj.length, setupObj.numParticles)

velocities = np.zeros((setupObj.numSteps, setupObj.numParticles))
initStreamVelocity = .3
velocities[0] = ((np.arange(setupObj.numParticles)%2)*2-1)*initStreamVelocity

totalMomentum = np.zeros(setupObj.numSteps)
totalKinEnergy = np.zeros(setupObj.numSteps)
totalPotEnergy = np.zeros(setupObj.numSteps)
totalEnergy = np.zeros(setupObj.numSteps)

Add small perturbations to the initial positions of particles to observe oscillatory modes in the plasma

In [ ]:
# Attempt adding normally-distributed perturbations to positions and velocities
posPerturb_stddev = setupObj.length/40
velPerturb_stddev = initStreamVelocity/20
positions[0] += np.random.randn(setupObj.numParticles) * posPerturb_stddev
velocities[0] += np.random.randn(setupObj.numParticles) * velPerturb_stddev

Main program loop (runs once for each simulation cycle, numSteps times)

In [ ]:
for it in range(setupObj.numSteps-1):
    positions[it] %= setupObj.length
    
    # Assign charge densities to cells (using nearest-neighbour approach)
    nearestCells = np.array(np.round(positions[it]/setupObj.cellSize), dtype=int) % setupObj.numCells
    np.add.at(chargedensities[it], nearestCells, 1)
    chargedensities[it] /= setupObj.cellSize
    
    # Calculate the potentials
    potSolverMatrix = (np.diag(np.full(setupObj.numCells,-2.))
                       + np.diag(np.ones(setupObj.numCells-1),1)
                       + np.diag(np.ones(setupObj.numCells-1),-1))                
#     potSolverMatrix[0,-1]=1
#     potSolverMatrix[-1,0]=1
    potentials[it] = la.solve(potSolverMatrix, -chargedensities[it]*(setupObj.cellSize)**2)
    
    # Calculate the electric fields
    elfieldSolver = np.concatenate([[potentials[it,-1]], potentials[it], [potentials[it,0]]])
    elfields[it] = ((-0.5/setupObj.cellSize)
                    * (elfieldSolver[2:2+setupObj.numCells]-elfieldSolver[:setupObj.numCells]))
    # Calculate the electric fields at particle positions
    elfields_atparticles = elfields[it, nearestCells]
    
    # Update velocities and positions (based on electric fields)
    velocities[it+1] = velocities[it] + setupObj.timeStep*setupObj.qm_ratio*elfields_atparticles
    positions[it+1] = positions[it] + setupObj.timeStep*velocities[it+1]

After the simulation has run we gather the positions and velocities of the particles over time, so now we can animate the scenario (creates a javascript in-line video player for the simulation)

In [ ]:
positions+=setupObj.length/2
positions%=setupObj.length
anim = animation.FuncAnimation(fig, 
                     func=animateStep, 
                     frames=np.arange(setupObj.numSteps), 
                     interval=40, # interval=1000/framerate 
                     blit=True)

HTML(anim.to_jshtml())
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)
# anim.save('TwoStreams_PICsimulation.mp4', writer=writer)

In [ ]:
%matplotlib inline
plt.plot(np.sum(velocities**2,axis=1))
1

In [ ]:
plt.close("all")

In [ ]:
def twoStreamPICsim(length=1.,
                    timeStep=0.1,
                    initStreamVelocity=0.1,
                    posPerturb_stddev=1/50,
                    velPerturb_stddev=.1/50,
                    qmRatio=1,
                    numSteps=100, 
                    numCells=50, 
                    numParticles=100,
                    color1='r',
                    color2='b',
                    displayOrSave=True,
                    fps=25):
    cellSize = length / numCells
    
    potentials = np.zeros((numSteps, numCells))
    elfields = np.zeros((numSteps, numCells))
    chargedensities = np.zeros((numSteps, numCells))

    positions = np.zeros((numSteps, numParticles))
    positions[0] = np.linspace(0, length, numParticles)
    positions[0] += np.random.randn(numParticles) * posPerturb_stddev
    velocities = np.zeros((numSteps, numParticles))
    velocities[0] = ((np.arange(numParticles)%2)*2-1)*initStreamVelocity
    velocities[0] += np.random.randn(numParticles) * velPerturb_stddev
    
    totalMomentum = np.zeros(numSteps)
    totalKinEnergy = np.zeros(numSteps)
    totalPotEnergy = np.zeros(numSteps)
    totalEnergy = np.zeros(numSteps)
    
    runSimulation(positions, velocities, potentials, elfields, chargedensities,
                  length, timeStep, cellSize, qmRatio, numSteps, numCells, numParticles)
    
    animateSimulation(positions, velocities, numParticles, length, initStreamVelocity, 
                      color1, color2, displayOrSave, fps, numSteps)

In [ ]:
def runSimulation(positions, velocities, 
                  potentials, elfields, chargedensities,
                  length, timeStep, cellSize, qmRatio, 
                  numSteps, numCells, numParticles):
    for it in range(numSteps-1):
        positions[it] %=  length

        # Assign charge densities to cells (using nearest-neighbour approach)
        nearestCells = np.array(np.round(positions[it]/ cellSize), dtype=int) % numCells
        np.add.at(chargedensities[it], nearestCells, 1)
        chargedensities[it] /= cellSize

        # Calculate the potentials
        potSolverMatrix = (np.diag(np.full(numCells,-2.))
                           + np.diag(np.ones(numCells-1),1)
                           + np.diag(np.ones(numCells-1),-1))                
    #     potSolverMatrix[0,-1]=1
    #     potSolverMatrix[-1,0]=1
        potentials[it] = la.solve(potSolverMatrix, -chargedensities[it]*(cellSize)**2)

        # Calculate the electric fields
        elfieldSolver = np.concatenate([[potentials[it,-1]], potentials[it], [potentials[it,0]]])
        elfields[it] = ((-0.5/cellSize)
                        * (elfieldSolver[2:2+numCells]-elfieldSolver[:numCells]))
        # Calculate the electric fields at particle positions
        elfields_atparticles = elfields[it, nearestCells]

        # Update velocities and positions (based on electric fields)
        velocities[it+1] = velocities[it] + timeStep*qmRatio*elfields_atparticles
        positions[it+1] = positions[it] + timeStep*velocities[it+1]

In [ ]:
def animateSimulation(positions, velocities, numParticles, length, initStreamVelocity, 
                      color1, color2, displayOrSave, fps, numSteps):
    mpl.rcParams.update(mpl.rcParamsDefault)
    mpl.rcParams['animation.embed_limit'] = 2**64

    fig, ax = plt.subplots()
    colors = np.array([(color1 if i%2==0 else color2) for i in range(numParticles)])
    scatterPlot = ax.scatter(0, 0, s=4.)

    ax.set_xlim(0, length)
    ax.set_ylim(-initStreamVelocity*8, initStreamVelocity*8)
    dynamicAxisLimits = False
    axisLimitWspace = 1.2
    minVelocities = np.min(velocities, axis=1) * axisLimitWspace
    minVelocities = minVelocities * (minVelocities<0)
    maxVelocities = np.max(velocities, axis=1) * axisLimitWspace
    maxVelocities = maxVelocities * (maxVelocities>0)

    def animateStep(frame):
        scatterPlot.set_offsets(np.vstack([positions[frame],velocities[frame]]).T)
        scatterPlot.set_color(colors)
        if dynamicAxisLimits and frame%10==0:
            ax.set_ylim(minVelocities[frame], maxVelocities[frame])
        return scatterPlot,
    
    positions+=length/2
    positions%=length
    anim = animation.FuncAnimation(fig, 
                         func=animateStep, 
                         frames=np.arange(numSteps), 
                         interval=1000/fps,
                         blit=True)

    if displayOrSave:
        display(HTML(anim.to_jshtml()))
    else:       
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)
        anim.save('TwoStreams_PICsimulation.mp4', writer=writer)

In [ ]:
twoStreamPICsim(length=1.,
                timeStep=0.06,
                initStreamVelocity=0.4,
                posPerturb_stddev=0.05,
                velPerturb_stddev=0,
                qmRatio=.002,
                numSteps=500, 
                numCells=70, 
                numParticles=600,
                color1='r',
                color2='b',
                displayOrSave=True,
                fps=20)